In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from IPython.display import clear_output

In [3]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-ebggkdt6
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-ebggkdt6
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=36229 sha256=8872b142a54500a8d83ac114da10a8dacb009261cbc015a9fffb5e5646313b6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-iqud6fzg/wheels/b7/17/95/4b16fafe1b84fdabd247eb5f01b41165bd98d4c82d64ab93d2
Successfully built autoattack


In [4]:
! git clone https://github.com/fra31/auto-attack.git

Cloning into 'auto-attack'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 672 (delta 162), reused 159 (delta 141), pack-reused 463
Receiving objects: 100% (672/672), 40.63 MiB | 13.41 MiB/s, done.
Resolving deltas: 100% (413/413), done.


In [5]:
! git clone https://github.com/hojat-ardi/image-project.git

Cloning into 'image-project'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 296 (delta 60), reused 135 (delta 40), pack-reused 111
Receiving objects: 100% (296/296), 39.81 MiB | 16.58 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [6]:
!cp /content/image-project/ImageProject-master/phase3/Eliminating_of_average_pooling/resnet_Eliminating_of_average_pooling.py  /content/auto-attack/autoattack/examples

# AA-L2

In [7]:
%%shell
cat > /content/auto-attack/autoattack/examples/eval.py << EOF


import os
import argparse
from pathlib import Path
import warnings

import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms

import sys
sys.path.insert(0,'..')
from resnet_Eliminating_of_average_pooling import *

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='./data')
    parser.add_argument('--norm', type=str, default='L2')
    parser.add_argument('--epsilon', type=float, default=0.3)
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--model', type=str, default='/content/gdrive/MyDrive/Eliminating_avgP_ResNet18.pth')
    #///////////////////////////////////////////////////////////////
    parser.add_argument('--n_ex', type=int, default=1000)
    parser.add_argument('--individual', action='store_true')
    parser.add_argument('--save_dir', type=str, default='./results')
    parser.add_argument('--batch_size', type=int, default=500)
    parser.add_argument('--log_path', type=str, default='./log_file.txt')
    parser.add_argument('--version', type=str, default='standard')
    parser.add_argument('--state-path', type=Path, default=None)

    args = parser.parse_args()

    # load model
    model = ResNet18()
    #//////////////////////////////////////////////
    model = nn.DataParallel(model)
    ckpt = torch.load(args.model)
    model.load_state_dict(ckpt['net'])
    #//////////////////////////////////////////////
    model.cuda()
    model.eval()

    # load data
    Normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    transform_list = [transforms.ToTensor() , Normalize]
    transform_chain = transforms.Compose(transform_list)
    item = datasets.CIFAR10(root=args.data_dir, train=False, transform=transform_chain, download=True)
    test_loader = data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)

    # create save dir
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    # load attack
    from autoattack import AutoAttack
    adversary = AutoAttack(model, norm=args.norm, eps=args.epsilon, log_path=args.log_path,
        version=args.version)

    l = [x for (x, y) in test_loader]
    x_test = torch.cat(l, 0)
    l = [y for (x, y) in test_loader]
    y_test = torch.cat(l, 0)

    # example of custom version
    if args.version == 'custom':
        adversary.attacks_to_run = ['apgd-ce', 'fab']
        adversary.apgd.n_restarts = 2
        adversary.fab.n_restarts = 2

    # run attack and save images
    with torch.no_grad():
        if not args.individual:
            adv_complete = adversary.run_standard_evaluation(x_test[:args.n_ex], y_test[:args.n_ex],
                bs=args.batch_size, state_path=args.state_path)

            torch.save({'adv_complete': adv_complete}, '{}/{}_{}_1_{}_eps_{:.5f}.pth'.format(
                args.save_dir, 'aa', args.version, adv_complete.shape[0], args.epsilon))

        else:
            # individual version, each attack is run on all test points
            adv_complete = adversary.run_standard_evaluation_individual(x_test[:args.n_ex],
                y_test[:args.n_ex], bs=args.batch_size)

            torch.save(adv_complete, '{}/{}_{}_individual_1_{}_eps_{:.5f}_plus_{}_cheap_{}.pth'.format(
                args.save_dir, 'aa', args.version, args.n_ex, args.epsilon))



/bin/bash: line 89: warning: here-document at line 0 delimited by end-of-file (wanted `EOF')


## Epsilon = 0.3

In [8]:
epsilon = 0.3
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon

100% 170498071/170498071 [00:13<00:00, 12827486.33it/s]
Extracting ./data/cifar-10-python.tar.gz to ./data
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 93.80%
apgd-ce - 1/2 - 200 out of 500 successfully perturbed
apgd-ce - 2/2 - 188 out of 438 successfully perturbed
robust accuracy after APGD-CE: 55.00% (total time 53.3 s)
apgd-t - 1/2 - 7 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 50 successfully perturbed
robust accuracy after APGD-T: 54.30% (total time 345.9 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 43 successfully perturbed
robust accuracy after FAB-T: 54.30% (total time 892.6 s)
square - 1/2 - 4 out of 500 successfully perturbed
square - 2/2 - 0 out of 43 successfully perturbed
robust accuracy after SQUARE: 53.90% (total time 1636.9 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 53.90%


## Epsilon = 0.4

In [9]:
epsilon = .4
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 93.80%
apgd-ce - 1/2 - 201 out of 500 successfully perturbed
apgd-ce - 2/2 - 190 out of 438 successfully perturbed
robust accuracy after APGD-CE: 54.70% (total time 55.7 s)
apgd-t - 1/2 - 6 out of 500 successfully perturbed
apgd-t - 2/2 - 0 out of 47 successfully perturbed
robust accuracy after APGD-T: 54.10% (total time 354.1 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 41 successfully perturbed
robust accuracy after FAB-T: 54.10% (total time 899.4 s)
square - 1/2 - 3 out of 500 successfully perturbed
square - 2/2 - 0 out of 41 successfully perturbed
robust accuracy after SQUARE: 53.80% (total time 1659.1 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 53.80%


## Epsilon = 0.5

In [10]:
epsilon = 0.5
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 93.80%
apgd-ce - 1/2 - 202 out of 500 successfully perturbed
apgd-ce - 2/2 - 188 out of 438 successfully perturbed
robust accuracy after APGD-CE: 54.80% (total time 55.7 s)
apgd-t - 1/2 - 10 out of 500 successfully perturbed
apgd-t - 2/2 - 1 out of 48 successfully perturbed
robust accuracy after APGD-T: 53.70% (total time 349.0 s)
fab-t - 1/2 - 0 out of 500 successfully perturbed
fab-t - 2/2 - 0 out of 37 successfully perturbed
robust accuracy after FAB-T: 53.70% (total time 888.9 s)
square - 1/2 - 2 out of 500 successfully perturbed
square - 2/2 - 0 out of 37 successfully perturbed
robust accuracy after SQUARE: 53.50% (total time 1636.7 s)
max L2 perturbation: 110.26619, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 53.50%


# AA-Linf

## Epsilon = 4/255

In [15]:
norm = "Linf"
epsilon = 4/255
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 93.80%
apgd-ce - 1/2 - 340 out of 500 successfully perturbed
apgd-ce - 2/2 - 304 out of 438 successfully perturbed
robust accuracy after APGD-CE: 29.40% (total time 52.8 s)
apgd-t - 1/1 - 0 out of 294 successfully perturbed
robust accuracy after APGD-T: 29.40% (total time 210.4 s)
fab-t - 1/1 - 0 out of 294 successfully perturbed
robust accuracy after FAB-T: 29.40% (total time 497.6 s)
square - 1/1 - 0 out of 294 successfully perturbed
robust accuracy after SQUARE: 29.40% (total time 867.3 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 29.40%


## Epsilon = 6/255

In [18]:
epsilon = 6/255
norm = "Linf"
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 93.80%
apgd-ce - 1/2 - 394 out of 500 successfully perturbed
apgd-ce - 2/2 - 345 out of 438 successfully perturbed
robust accuracy after APGD-CE: 19.90% (total time 55.7 s)
apgd-t - 1/1 - 0 out of 199 successfully perturbed
robust accuracy after APGD-T: 19.90% (total time 164.5 s)
fab-t - 1/1 - 0 out of 199 successfully perturbed
robust accuracy after FAB-T: 19.90% (total time 362.0 s)
square - 1/1 - 0 out of 199 successfully perturbed
robust accuracy after SQUARE: 19.90% (total time 615.9 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 19.90%


## Epsilon = 8/255

In [19]:
epsilon = 8/255
norm = "Linf"
! python /content/auto-attack/autoattack/examples/eval.py --epsilon $epsilon --norm $norm

Files already downloaded and verified
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 93.80%
apgd-ce - 1/2 - 435 out of 500 successfully perturbed
apgd-ce - 2/2 - 386 out of 438 successfully perturbed
robust accuracy after APGD-CE: 11.70% (total time 55.8 s)
apgd-t - 1/1 - 2 out of 117 successfully perturbed
robust accuracy after APGD-T: 11.50% (total time 112.9 s)
fab-t - 1/1 - 0 out of 115 successfully perturbed
robust accuracy after FAB-T: 11.50% (total time 223.8 s)
square - 1/1 - 0 out of 115 successfully perturbed
robust accuracy after SQUARE: 11.50% (total time 377.8 s)
max Linf perturbation: 2.42907, nan in tensor: 0, max: 2.75373, min: -2.42907
robust accuracy: 11.50%
